# Exercícios do curso de Pandas avançado

## Começando o trabalho

### Carregando arquivos JSON

Em uma base de dados que relaciona o nome da pessoa e seus dados referentes à idade, peso e altura, considere a variável data_json abaixo:

In [1]:
import pandas as pd
data_json = '{"Rita": {"Idade": 24, "Peso": 62, "Altura": 1.65}, "Zeca": {"Idade": 32, "Peso": 80, "Altura": 1.82}}'

Indique a opção que mostra os códigos que produzem os seguintes resultados:
    
**Tabela A**

| | Idade | Peso | Altura |
|---|---|---|---|
|Rita | 24 | 62 | 1.65 |
|Zeca | 32 | 80 | 1.82 |

**Tabela B**

|	|Rita|	Zeca|
|---|---|---|
|Idade|	24|	32|
|Peso	|62|	80|
|Altura	|1.65|	1.82|

In [2]:
dados = pd.read_json(
    path_or_buf = data_json,
    orient = 'index'
)
dados

,Idade,Peso,Altura
Rita,24,62,1.65
Zeca,32,80,1.82


In [3]:
dados = pd.read_json(
    path_or_buf = data_json
)
dados

,Rita,Zeca
Idade,24.00,32.00
Peso,62.00,80.00
Altura,1.65,1.82


### Carregando arquivos Excel

No arquivo Excel (XLSX) disponibilizado (bairros.xlsx) temos duas planilhas. Na planilha “Residencial X Comercial”, temos os valores do m2 para imóveis comerciais e residenciais nos bairros da cidade do Rio de Janeiro. Temos também a divisão regional das informações por zonas da cidade.

Utilizando esta planilha e o método `read_excel()` do pandas obtenha o seguinte DataFrame:

|Zonas	|Bairros|	Residencial|	Comercial|
|---|---|---|---|
|Sul	|Botafogo	|14002	|7972|
||Catete	|15232|	6259|
||Copacabana	|23318|	9355|
|... |...|...|...|

Observe que foram selecionadas apenas as linhas dos bairros pertencentes à zona sul da cidade.

Marque a opção que apresenta o código necessário para criar o DataFrame acima.

In [4]:
pd.read_excel(
    io = 'dados/bairros.xlsx',
    sheet_name = 'Residencial X Comercial',
    #header = 1,
    names = ['Zonas', 'Bairros', 'Residencial', 'Comercial'],
    index_col = [0, 1],
    usecols = 'B:E',
    skiprows = 18,
    nrows = 17
)

Residencial  Comercial
Zonas Bairros                                
Sul   Botafogo               14002       7972
      Catete                 15232       6259
      Copacabana             23318       9355
      Cosme Velho            10320       8177
      Flamengo               19636       7135
      Gávea                  13506       8211
      Humaitá                10603       5039
      Ipanema                15965       7293
      Jardim Botânico        17243       8095
      Lagoa                  24982       6584
      Laranjeiras            17307       7745
      Leblon                 13815       8125
      Leme                   15815       8587
      Rocinha                15110       9023
      São Conrado            20727       9587
      Urca                   22876       6607
      Vidigal                23470       5424

## Transformando e tratando os dados

### Normalizando nossos dados

Estamos trabalhando com uma base de dados que relaciona o nome dos alunos, suas idades e medidas (peso e altura). Considere a variável data_json abaixo:

In [5]:
data_json = '{"alunos": [{"Nome": "Rita", "Info": {"Idade": 24, "Medidas": {"Peso": 62, "Altura": 1.65}}}, {"Nome": "Zeca", "Info": {"Idade": 32, "Medidas": {"Peso": 80, "Altura": 1.82}}}]}'

Utilizando os métodos aprendidos, assinale a alternativa que produz como resultado o seguinte DataFrame:


||Nome	|Info_Idade|	Info_Medidas|
|---|---|---|---|
|0	|Rita|	24|	{'Peso': 62, 'Altura': 1.65}|
|1|	Zeca|	32|	{'Peso': 80, 'Altura': 1.82}|


In [6]:
df_json = pd.read_json(data_json)
df_json

,alunos
0,"{'Nome': 'Rita', 'Info': {'Idade': 24, 'Medida..."
1,"{'Nome': 'Zeca', 'Info': {'Idade': 32, 'Medida..."


In [7]:
pd.json_normalize(data = df_json.alunos, sep = '_', max_level = 1)

,Nome,Info_Idade,Info_Medidas
0,Rita,24,"{'Peso': 62, 'Altura': 1.65}"
1,Zeca,32,"{'Peso': 80, 'Altura': 1.82}"


### Praticando o uso de métodos de strings

Em algumas bases de dados encontramos colunas de informações no formato de texto (strings) que podem apresentar, em seu conteúdo, mais de um tipo de informação. Nestes casos torna-se necessário utilizar métodos específicos para tentar separar estas informações em colunas distintas. Vamos treinar a utilização destes métodos com a variável data_string abaixo:

In [8]:
data_string = "#-> Churrasqueira | Sauna | Mobiliado | Piscina <-#"

Utilizando os métodos de string que aprendemos no último vídeo, assinale os itens que retornam o seguinte resultado:

`['Churrasqueira', 'Sauna', 'Mobiliado', 'Piscina']`

In [9]:
data_string[4:-4].split(' | ')

['Churrasqueira', 'Sauna', 'Mobiliado', 'Piscina']

In [10]:
data_string.strip('#->< ').split(' | ')

['Churrasqueira', 'Sauna', 'Mobiliado', 'Piscina']

### Utilizando o método filter

Em algumas situações, durante a fase de exploração dos dados, o profissional de data science precisa executar alguns filtros no dataset. Este procedimento pode ter como objetivo a aplicação de certos tipos de tratamento em um grupo específico de colunas ou a geração de tabulações específicas entre certas variáveis.

O método `filter` que vimos possibilita a criação de subconjunto das linhas ou colunas de um DataFrame de acordo com os rótulos dos eixos especificados. Observe que este método não filtra um DataFrame em seu conteúdo. O filtro é apenas aplicado aos rótulos dos índices ou colunas.

Considere o DataFrame `df`:

In [11]:
import pandas as pd
dados = {
    "alunos": ["Rita", "Lucas", "Zeca", "Ana"], 
    "idade": [10, 12, 11, 10], 
    "medidas_altura": [1.3, 1.5, 1.45, 1.28], 
    "medidas_peso": [42, 50, 45, 38]
}

df = pd.DataFrame(dados)

Utilizando o método `filter`, assinale a opção que retorna um DataFrame com apenas as colunas "medidas_altura" e "medidas_peso".

In [12]:
df.filter(like = 'medidas')

,medidas_altura,medidas_peso
0,1.30,42
1,1.50,50
2,1.45,45
3,1.28,38


## Combinando conjuntos de dados

### Os métodos append e concat

Em certos momentos, durante a fase de exploração dos dados, precisamos juntar em nosso dataset novas informações. Isso é bastante comum em projetos de data science que precisam ser atualizados constantemente devido a ocorrência de novos registros sobre o objeto de estudo. Para este tipo de trabalho o pandas disponibiliza alguns métodos como o `append` e o `concat` que conhecemos.

Assinale a alternativa que apresenta a forma correta de se utilizar os métodos `append` e `concat` do `pandas` para obter o seguinte resultado:


||A|	B|
|---|---|---|
|0|	1|	1|
|1|	2|	4|
|2|	3|	9|
|3|	4|	16|
|4|	5|	25|
|5|	6|	36|

Para isso utilize os dois DataFrames resultantes do código abaixo:

In [13]:
import pandas as pd
df_A = pd.DataFrame({'A': [1, 2, 3], "B": [1, 4, 9]})
df_B = pd.DataFrame({'A': [4, 5, 6], "B": [16, 25, 36]})

In [14]:
pd.concat([df_A, df_B], ignore_index = True)

,A,B
0,1,1
1,2,4
2,3,9
3,4,16
4,5,25
5,6,36


In [15]:
df_A.append(df_B, ignore_index = True)

C:\Users\beatr\AppData\Local\Temp\ipykernel_2920\1994273176.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_A.append(df_B, ignore_index = True)


,A,B
0,1,1
1,2,4
2,3,9
3,4,16
4,5,25
5,6,36


### O parâmetro sort dos métodos append e concat

Os métodos `append` e `concat` possuem o parâmetro `sort` que é um booleano que vem, por *padrão*, configurado como `False`. Quando configurado como `True` classifica os eixos (colunas no caso do `append` e colunas ou linhas no caso do `concat`) caso ainda não estejam alinhados.

Para entender melhor como essa classificação funciona vamos a um exemplo prático. Considere os DataFrames **df_A** e **df_B**:

In [16]:
df_A = pd.DataFrame({'A': [1, 2, 3], 'B': [1, 4, 9]})

In [17]:
df_B = pd.DataFrame({'B': [16, 25, 36], 'A': [4, 5, 6]})

Executando a linha de código abaixo, qual seria o DataFrame resultante?

In [18]:
df_B.append(df_A, ignore_index=True, sort=True)

C:\Users\beatr\AppData\Local\Temp\ipykernel_2920\3686226822.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_B.append(df_A, ignore_index=True, sort=True)


,A,B
0,4,16
1,5,25
2,6,36
3,1,1
4,2,4
5,3,9


### Substituindo valores

A Series nomes contém os nomes de alguns assuntos interessantes quando falamos sobre análise de dados:

In [19]:
nomes = pd.Series(['Data Science', 'Big Data', 'DS', 'Machine Learning', 'ML'])
nomes

0        Data Science
1            Big Data
2                  DS
3    Machine Learning
4                  ML
dtype: object

Assinale os códigos que, com os métodos vistos, geram a Series abaixo:

||0|
|---|---|
|0|	Data Science|
|1|	Big Data|
|2|	Machine Learning|

Observe que 'DS' e 'ML' são siglas para 'Data Science' e 'Machine Learning', respectivamente.

In [20]:
pd.Series(nomes.replace({'DS': 'Data Science', 'ML': 'Machine Learning'}).unique())

0        Data Science
1            Big Data
2    Machine Learning
dtype: object

In [21]:
# outras possibilidades (colocadas como alternativas do exercício)
pd.Series(nomes.replace(['DS', 'ML'], ['Data Science', 'Machine Learning']).unique())

0        Data Science
1            Big Data
2    Machine Learning
dtype: object

In [22]:
pd.Series(nomes.replace('DS', 'Data Science').replace('ML', 'Machine Learning').unique())

0        Data Science
1            Big Data
2    Machine Learning
dtype: object

### Tipos de junção

Utilize os DataFrames df_1 e df_2 para responder esta questão.

In [23]:
df_1 = pd.DataFrame([2, 4, 6, 8], index=['A', 'B', 'C', 'D'], columns=['pares'])
df_2 = pd.DataFrame([1, 3, 5, 7], index=['C', 'D', 'E', 'F'], columns=['impares'])

Utilizando como base o código abaixo, assinale a alternativa que apresenta a configuração correta do parâmetro how para obter como resultado os três DataFrames abaixo.

```
pd.merge(
    left=df_1, 
    right=df_2,
    left_index=True,
    right_index=True,
    how='????'
)
```

||pares|	impares|
|---|---|---|
|C|	6|	1|
|D|	8|	3|

In [24]:
pd.merge(
    left=df_1, 
    right=df_2,
    left_index=True,
    right_index=True,
    how='inner'
)

,pares,impares
C,6,1
D,8,3


II)

||pares|	impares|
|---|---|---|
|A|	2|	nan|
|B|	4|	nan|
|C|	6|	1|
|D|	8|	3|

In [25]:
pd.merge(
    left=df_1, 
    right=df_2,
    left_index=True,
    right_index=True,
    how='left'
)

,pares,impares
A,2,NaN
B,4,NaN
C,6,1.0
D,8,3.0


III)

||pares|	impares|
|---|---|---|
|A|	2|	nan|
|B|	4|	nan|
|C|	6|	1|
|D|	8|	3|
|E|	nan|	5|
|F|	nan|	7|

In [26]:
pd.merge(
    left=df_1, 
    right=df_2,
    left_index=True,
    right_index=True,
    how='outer'
)

,pares,impares
A,2.0,NaN
B,4.0,NaN
C,6.0,1.0
D,8.0,3.0
E,NaN,5.0
F,NaN,7.0


## Adicionando informações

### Desafio Regex

Vimos como extrair informações de dados no formato *string* com o uso do método `extractall` e expressões regulares simples.

Expressões regulares são *strings* que descrevem um padrão de pesquisa que pode ser utilizado para combinar ou substituir padrões dentro de uma *string* com uma quantidade mínima de esforço. Podem ser utilizadas em tarefas de procura, substituição, validação e filtragem de *strings*.

Usamos a seguinte expressão para extrair de um conjunto de strings os seus valores numéricos: `(\d+)`

Os parênteses (`()`) são utilizados para definir um grupo dentro da expressão, o (`\d`) indica que procuramos por dígitos numéricos de 0 a 9 e o sinal de mais (`+`) significa "corresponder a um ou mais" da expressão anterior a ele. Mas imagine que agora estamos procurando por números de CEP ou números telefônicos dentro de sequências de texto. Considere a seguinte Series com informações de um grupo de pessoas:

In [27]:
pessoas = pd.Series([
    'Nome: Mariana Sousa | End.: Rua Damasco, 1978 Japeri-RJ 26.473-790 | Tel.: (21) 99131-8473',
    'Nome: Aline Cardoso | End.: Rua Paschoal Marmirolli, 577 Sumaré-SP 13.171-700 | Tel.: (19) 8577-4777',
    'Nome: Vitór Fernandes | Tel.: (21) 5923-5723 | End.: Rua Bernardo Franco, 1520 São Gonçalo-RJ 24.470-190',
    'Nome: Victor Sousa | Tel.: (11) 98618-2626 | End.: Rua Santa Terezinha, 27 Suzano-SP 08.694-410',
    'Nome: Vitória Dias | Tel.: (19) 97632-5829 | End.: Rua 4 JA, 500 Rio Claro-SP 13.506-010',
    'Nome: Douglas Santos | Tel.: (11) 4890-8192 | End.: Rua Alcides Teodoro Santos, 1268 São Paulo-SP 05.762-010',
    'Nome: Kauan Pinto | Tel.: (14) 6752-6858 | End.: Rua Amélia Volta Laplechade, 1334 Marília-SP 17.511-801',
    'Nome: Miguel Silva | End.: Rua Severina Ferreira, 1408 João Pessoa-PB 58.034-160 | Tel.: (83) 7077-6476',
    'Nome: Luis Castro | Tel.: (31) 97711-4493 | End.: Rua São Bento, 969 Ribeirão das Neves-MG 33.930-290',
    'Nome: Thiago Almeida | Tel.: (18) 2031-3622 | End.: Rua Liberdade, 669 Araçatuba-SP 16.015-425'
])

Note que cada registro da *Series* `pessoas` contém uma *string* com as informações de nome, telefone e endereço das pessoas, e que estas informações não se encontram sempre nesta ordem, exceto pela informação sobre o nome.

Observe que na informação sobre o endereço de cada pessoa temos o CEP que vem no seguinte formato: `XX.XXX-XXX`. Vamos construir uma expressão regular para extrair essa informação das strings da Series `pessoas`.

In [28]:
pessoas.str.extractall('(\d{2}.\d{3}-\d{3})')

,,0
,match,
0,0,26.473-790
1,0,13.171-700
2,0,24.470-190
3,0,08.694-410
4,0,13.506-010
5,0,05.762-010
6,0,17.511-801
7,0,58.034-160
8,0,33.930-290


Veja que você também pode especificar o número exato de correspondências com o uso de números dentro da chaves:

 * `{n}`: exatamente n ocorrências
 * `{n,}`: n ou mais ocorrências
 * `{,m}`: no máximo m ocorrências
 * `{n,m}`: entre n e m ocorrências

No código anterior, `\d{2}` indica que naquela posição devemos ter exatamente dois dígitos numéricos.

Outro ponto que deve ser observado quando criamos uma expressão regular é que quando precisamos especificar caracteres dentro de uma string que já são utilizados como operadores dentro de uma expressão regular, como por exemplo os parênteses, precisamos utilizar o caractere de escape (`\`) antes para denotar que o respectivo caractere não se trata de um operador da expressão e sim um caractere que desejamos extrair da *string*.

Muito bem, hora de colocar a mão na massa. Observe que as *strings* da *Series* `pessoas` também apresentam as informações de números de telefones. Crie uma expressão regular para extrair estes números. Note que alguns números têm tamanhos diferentes (celulares e fixos).

Assinale a resposta com o código correto.

Dica: Em expressões regulares, \s representa espaço em branco.

In [30]:
numeros = pessoas.str.extractall('(\(\d{2}\)\s\d{4,5}\-\d{4})')
numeros

,,0
,match,
0,0,(21) 99131-8473
1,0,(19) 8577-4777
2,0,(21) 5923-5723
3,0,(11) 98618-2626
4,0,(19) 97632-5829
5,0,(11) 4890-8192
6,0,(14) 6752-6858
7,0,(83) 7077-6476
8,0,(31) 97711-4493


In [56]:
enderecos = pessoas.str.extractall('([\w+\s]*\,\s\d{,9}\s[\w+\s]*\-\w{2})')
enderecos

,,0
,match,
0,0,"Rua Damasco, 1978 Japeri-RJ"
1,0,"Rua Paschoal Marmirolli, 577 Sumaré-SP"
2,0,"Rua Bernardo Franco, 1520 São Gonçalo-RJ"
3,0,"Rua Santa Terezinha, 27 Suzano-SP"
4,0,"Rua 4 JA, 500 Rio Claro-SP"
5,0,"Rua Alcides Teodoro Santos, 1268 São Paulo-SP"
6,0,"Rua Amélia Volta Laplechade, 1334 Marília-SP"
7,0,"Rua Severina Ferreira, 1408 João Pessoa-PB"
8,0,"Rua São Bento, 969 Ribeirão das Neves-MG"


### Mais alguns métodos de strings

Junto com o atributo `str` de um *Series* também é possível utilizar outros métodos de tratamento de strings bastante úteis em projetos de *data science*.

Os métodos `upper` e `lower` são bons exemplos de métodos úteis durante procedimentos de comparação envolvendo strings. O método `upper` converte todos os caracteres de uma *string* para maiúsculo e o método `lower` faz o contrário.

Outro método bastante útil é o `contains`. Com este método podemos testar se determinado trecho de texto está contido em uma *string* dentro de uma *Series*. No método `contains` também podemos utilizar expressões regulares (`default`) e também passar apenas trechos de texto, bastando para isso configurar o parâmetro `regex` como `False`.

Feitas estas considerações, considere a Series anuncios abaixo para responder a questão:

In [57]:
anuncios = pd.Series([
    "Amplo apartamento com vista para o mar, piscina, sauna e 2 vagas de garagem. CEP 22790-735",
    "Ótima oportunidade no Leblon! Casa de condomínio com 800m² próximo a praia.",
    "Sala e quarto em Copacabana. Próximo ao metrô.",
    "Venha morar na melhor localização do Rio de Janeiro. Piscina, academia e toda estrutura de lazer.",
    "Sala comercial no Centro da cidade. 23456-021",
    "Venha conhecer o melhor de Ipanema. A duas quadras da praia. Conheça o apartamento mobiliado.",
    "Melhor localização da Barra. Condomínio com piscina, academia, espaço gourmet e muito mais.",
    "Vende-se terreno em Pedra de Guaratiba. Tratar direto com o proprietário.",
    "Apartamento de alto padrão na Lagoa. Vista pro mar e sol da manhã.",
    "Passo o ponto de loja no centro da cidade (Saara). Ligar para (21) 1234-5678 e falar com Juca."
])

Utilizando as informações acima, marque as opções que apresentam os códigos capazes de identificar os três registros que contêm a palavra piscina.

In [61]:
anuncios[anuncios.str.contains('[p|P]iscina')]

0    Amplo apartamento com vista para o mar, piscin...
3    Venha morar na melhor localização do Rio de Ja...
6    Melhor localização da Barra. Condomínio com pi...
dtype: object

### Classificando alunos

O método cut, que conhecemos no último vídeo, é bastante útil na hora de criar classes a partir de dados numéricos. Para este exercício veja o DataFrame df abaixo:

In [62]:
df = pd.DataFrame(
    {
        'alunos': ['Juca', 'Zeca', 'Ana', 'Rita', 'Lia', 'Beto'],
        'notas': [8, 6.5, 10, 7, 6, 0]
    }
).set_index('alunos')

Este DataFrame tem as informações de notas finais de um conjunto de alunos. Utilizando o método cut crie uma coluna no DataFrame df que classifique estes alunos da seguinte maneira:

|notas|	resultado|
|---|---|
|De 0 até 5.9|	Reprovado|
|De 6 até 6.9	|Recuperação|
|De 7 até 10	|Aprovado|

Considere apenas uma casa decimal para as notas.

Marque a alternativa que apresenta o código correto.

In [65]:
classes = [0, 5.9, 6.9, 10]
rotulos = ['Reprovado', 'Recuperação', 'Aprovado']

df['resultado'] = pd.cut(x = df['notas'], bins = classes, labels = rotulos, include_lowest = True)
df

,notas,resultado
alunos,,
Juca,8.0,Aprovado
Zeca,6.5,Recuperação
Ana,10.0,Aprovado
Rita,7.0,Aprovado
Lia,6.0,Recuperação
Beto,0.0,Reprovado
